In [9]:
def convert_silver_to_gold():
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, current_timestamp
    import psycopg2
    import datetime

    nm_processo = "silver_to_gold_posicao"
    dt_delta = datetime.date.today()

    # === Spark Session ===
    spark = (
        SparkSession
        .builder
        .appName('pipeline_sptrans_silver_to_gold')
        .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
        .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
        .config("spark.hadoop.fs.s3a.access.key", "datalake")
        .config("spark.hadoop.fs.s3a.secret.key", "datalake")
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .getOrCreate()
    )

    # === PostgreSQL Connection Properties ===
    postgres_url = "jdbc:postgresql://db:5432/db"
    properties = {
        "user": "admin",
        "password": "admin",
        "driver": "org.postgresql.Driver"
    }

    # Conexão psycopg2 para log
    conn = psycopg2.connect(
        host="db",
        port=5432,
        database="db",
        user="admin",
        password="admin"
    )
    cur = conn.cursor()

    try:
        # === Lê dados Silver ===
        df_silver = spark.read \
            .format("jdbc") \
            .option("url", postgres_url) \
            .option("dbtable", "db.silver_sptrans.posicao") \
            .option("user", properties["user"]) \
            .option("password", properties["password"]) \
            .load()

        # === Transformações para Gold (sem cast) ===
        df_gold = df_silver.select(
            col("hr").alias("hr_referencia"),
            col("c").alias("cd_letreiro"),
            col("cl").alias("cd_linha"),
            col("sl").alias("nu_sentido"),
            col("lt1").alias("ds_destino"),
            col("lt0").alias("ds_origem"),
            col("qv").alias("qtde_veiculos_localizados"),
            col("p").alias("cd_prefixo_veiculo"),
            col("a").alias("flg_acessibilidade"),
            col("ta").alias("hr_localizacao"),
            col("py").alias("latitude"),
            col("px").alias("longitude"),            
            col("sv"),
            col("vs_is"),
            col("dt_delta"),
            current_timestamp().alias("dt_processamento")
        )

        # Conta quantidade de registros
        qtde_registro = df_gold.count()

        # === Grava dados Gold no PostgreSQL ===
        if qtde_registro > 0:
            df_gold.write \
                .mode("append") \
                .jdbc(url=postgres_url, table="db.gold_sptrans.posicao", properties=properties)

        # === Log de sucesso ===
        cur.execute(
            "INSERT INTO db.monitoramento_log.processos_log "
            "(nm_processo, dt_processamento, dt_delta, ds_status, ds_log, qtde_registro) "
            "VALUES (%s, CURRENT_TIMESTAMP, %s, %s, %s, %s)",
            (nm_processo, dt_delta, 1, "Processo executado com sucesso", qtde_registro)
        )
        conn.commit()

    except Exception as e:
        # === Log de falha com mensagem de erro ===
        cur.execute(
            "INSERT INTO db.monitoramento_log.processos_log "
            "(nm_processo, dt_processamento, dt_delta, ds_status, ds_log, qtde_registro) "
            "VALUES (%s, CURRENT_TIMESTAMP, %s, %s, %s, %s)",
            (nm_processo, dt_delta, 0, str(e)[:1000], 0)
        )
        conn.commit()
        print("Erro na conversão Silver -> Gold:", e)

    finally:
        cur.close()
        conn.close()
        spark.stop()


In [10]:
convert_silver_to_gold()
